In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
import joblib

# Step 1: Read the CSV file
df = pd.read_csv('output.csv')

df['date'] = pd.to_datetime(df['date'], dayfirst=True).map(pd.Timestamp.toordinal)
# Normalize the total carbon emission amounts
scaler = MinMaxScaler(feature_range=(0, 1))
df['total_carbon_emission'] = scaler.fit_transform(df[['total_carbon_emission']])

# Step 3: Split the data
X = df['date'].values.reshape(-1, 1)
y = df['total_carbon_emission'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Create the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(1,)),
    Dense(64, activation='relu'),
    Dense(1)
])

# Step 5: Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Step 6: Train the model
model.fit(X_train, y_train, epochs=110, batch_size=32, validation_split=0.2)

# Step 7: Evaluate the model
model.evaluate(X_test, y_test)

joblib.dump(model, 'carbon_emission_model.pkl')

Epoch 1/110
5/5 [==============================] - 1s 51ms/step - loss: 9233842176.0000 - val_loss: 3245078784.0000
Epoch 2/110
5/5 [==============================] - 0s 15ms/step - loss: 1558084480.0000 - val_loss: 45847624.0000
Epoch 3/110
5/5 [==============================] - 0s 17ms/step - loss: 141589344.0000 - val_loss: 608663232.0000
Epoch 4/110
5/5 [==============================] - 0s 16ms/step - loss: 823927360.0000 - val_loss: 903899840.0000
Epoch 5/110
5/5 [==============================] - 0s 12ms/step - loss: 715286656.0000 - val_loss: 353978176.0000
Epoch 6/110
5/5 [==============================] - 0s 10ms/step - loss: 182069280.0000 - val_loss: 9851154.0000
Epoch 7/110
5/5 [==============================] - 0s 11ms/step - loss: 13843098.0000 - val_loss: 63368384.0000
Epoch 8/110
5/5 [==============================] - 0s 12ms/step - loss: 95092912.0000 - val_loss: 115599520.0000
Epoch 9/110
5/5 [==============================] - 0s 14ms/step - loss: 93622048.0000 - val

['carbon_emission_model.pkl']

In [8]:
import pandas as pd
import numpy as np
import joblib
from datetime import datetime, timedelta

# Load the model and the pre-fitted scaler
model = joblib.load('carbon_emission_model.pkl')
#scaler = joblib.load('scaler.pkl')  # Load the pre-fitted scaler

# Get user input
month_year_str = input("Enter month and year (YYYY-MM): ")
year, month = map(int, month_year_str.split('-'))

# Generate all Wednesdays of the specified month
first_day = datetime(year, month, 1)
last_day = (first_day.replace(month=month % 12 + 1, day=1) - timedelta(days=1)).day
days = [first_day + timedelta(days=i) for i in range(last_day)]

# Prepare the data for prediction
day_ordinals = np.array([datetime.toordinal(day) for day in days]).reshape(-1, 1)
predicted_emissions = model.predict(day_ordinals)
predicted_emissions = scaler.inverse_transform(predicted_emissions)

# Create a DataFrame to store the results
results = pd.DataFrame({
    'Date': [day.strftime("%Y-%m-%d") for day in days],
    'Predicted Carbon Emission': predicted_emissions.flatten()
})

# Save the results to a CSV file
csv_filename = f"predicted_emissions_{year}_{month:02d}.csv"
results.to_csv(csv_filename, index=False)

print(f"Predicted carbon emissions saved to {csv_filename}")

Enter month and year (YYYY-MM): 2024-10
1/1 [==============================] - 0s 60ms/step
Predicted carbon emissions saved to predicted_emissions_2024_10.csv
